### Statistical analysis ###

In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt

### BPM ###

In [30]:
importlib.reload(dr_a)
importlib.reload(dt)


# what's the best plot for data visualization?

apd_metric_folder = dr_a.Paths.METRICS
phases = dr_a.Phases.PHASES_LIST
apd_mean_bpm = {}
for phase in phases:
    # print(f"PHASE: {phase} " + "-"*30)
    files = glob.glob(os.path.join(apd_metric_folder, f"bpm_{phase}*.csv"))

    dfs = [pd.read_csv(file, index_col=[0]).to_numpy() for file in files]
    for i in range(len(dfs)):
        df = dfs[i]
        subjects = pd.DataFrame(df[:, 0], columns=["subject"])
        mean_col = pd.DataFrame(np.nanmean(df[:, 1:], axis=1), columns=["mean_bpm"])
        mean_df = pd.concat([subjects, mean_col], axis=1)

        indices = mean_df.query('mean_bpm > 180').index.tolist()
        mean_df = mean_df.drop(indices, axis=0).reset_index()
        mean_df.dropna()

        dfs[i] = mean_df
    dfs = pd.concat(dfs, axis=0)
    apd_mean_bpm[phase] = dfs
    

wesad_metric_folder = dr_w.Paths.METRICS
phases = dr_w.Phases.PHASE_ORDER
wesad_mean_bpm = {}
for phase in phases:
    # print(f"PHASE: {phase} " + "-"*30)
    files = glob.glob(os.path.join(wesad_metric_folder, f"bpm_{phase}*.csv"))

    dfs = [pd.read_csv(file, index_col=[0]).to_numpy() for file in files]
    for i in range(len(dfs)):
        df = dfs[i]
        subjects = pd.DataFrame(df[:, 0], columns=["subject"])
        mean_col = pd.DataFrame(np.nanmean(df[:, 1:], axis=1), columns=["mean_bpm"])
        mean_df = pd.concat([subjects, mean_col], axis=1)

        indices = mean_df.query('mean_bpm > 180').index.tolist()
        mean_df = mean_df.drop(indices, axis=0).reset_index()
        mean_df = mean_df.dropna()

        dfs[i] = mean_df
    dfs = pd.concat(dfs, axis=0)
    wesad_mean_bpm[phase] = dfs
    

popane_metric_folder = dr_p.Paths.METRICS
studies = {
    "Study1": dr_p.Study1,
    "Study3": dr_p.Study3,
    "Study4": dr_p.Study4,
    "Study5": dr_p.Study5
}
popane_mean_bpm_1 = {}
popane_mean_bpm_3 = {}
popane_mean_bpm_4 = {}
popane_mean_bpm_5 = {}

popane_mean_bpm = [
    popane_mean_bpm_1,
    popane_mean_bpm_3,
    popane_mean_bpm_4,
    popane_mean_bpm_5
]
for i in range(len(popane_mean_bpm)):
    study = list(studies.keys())[i]
    phases = studies[study].ALL
    for phase in phases:
        # print(f"PHASE: {phase} " + "-"*30)

        files = glob.glob(os.path.join(popane_metric_folder, study, f"bpm_{phase}*.csv"))

        dfs = [pd.read_csv(file, index_col=[0]).to_numpy() for file in files]
        for i in range(len(dfs)):
            df = dfs[i]
            subjects = pd.DataFrame(df[:, 0], columns=["subject"])
            mean_col = pd.DataFrame(np.nanmean(df[:, 1:], axis=1), columns=["mean_bpm"])
            mean_df = pd.concat([subjects, mean_col], axis=1)

            indices = mean_df.query('mean_bpm > 180').index.tolist()
            mean_df = mean_df.drop(indices, axis=0).reset_index()
            mean_df = mean_df.dropna()

            dfs[i] = mean_df
        dfs = pd.concat(dfs, axis=0)
        popane_mean_bpm[i][phase] = dfs

C:\Users\zhoux\AppData\Local\Temp\ipykernel_7248\2734142962.py:41: RuntimeWarning: Mean of empty slice
  mean_col = pd.DataFrame(np.nanmean(df[:, 1:], axis=1), columns=["mean_bpm"])
